In [2]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 아이콘을 클릭하고
# 옵션에서 기간 선택하기
# 블로그 글, 링크 수집하기

In [14]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [15]:
# 크롤링 방법
# 1. requests, bs
# 2. v 셀레늄 
# 3. api 기반 

In [46]:
# 검색어
query_txt = "경기도 베이커리 카페"

In [47]:
# 크롬 드라이버 : 파이썬이 크롬 브라우저를 통제할 수 있게 도와주는 프로그램
# 크롬 드라이버를 '현재 작업 폴더'에 설치
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저
# driver : 크롬 브라우저

driver.get('https://www.naver.com/')
time.sleep(2)    # 2초간 정지 

In [48]:
# 네이버 검색창에 "검색어" 검색
selector = "#query"
element = driver.find_element(By.CSS_SELECTOR, selector)    # 검색창 찾기
element.clear()                # 현재 검색창에 무언가 들어있으면 삭제
element.send_keys(query_txt)   # 검색창에 검색어 입력

In [49]:
element.submit()    # 검색 버튼 클릭  
time.sleep(1) 

In [ ]:
# a 태그에 포함된 텍스트는 LINK_TEXT를 이용해서 텍스트를 셀렉터로 사용할 수 있다.

In [50]:
# '블로그' 클릭
driver.find_element(By.LINK_TEXT, "블로그").click( )
time.sleep(1)

In [51]:
# '옵션' 클릭
driver.find_element(By.LINK_TEXT, "옵션").click( )
time.sleep(1)

In [52]:
# LINK_TEXT를 이용한 검색기간 '3개월' 클릭
driver.find_element(By.LINK_TEXT, "1주").click( )
time.sleep(1)

In [53]:
# # CSS SELCTOR를 이용한 "3개월" 클릭
# selector = "#snb > div.mod_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(6)"
# driver.find_element(By.CSS_SELECTOR, selector).click()
# time.sleep(1)

In [54]:
# # 현재 페이지를 스크롤 다운하는 자바스크립트 코드
# driver.execute_script("window.scrollTo(0, 99999999)")
# time.sleep(1)

In [56]:
# 글 개수 확인
def blog_cnt():
    # 글 CSS SELECTOR 
    selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
    blog_cnt = len(driver.find_elements(By.CSS_SELECTOR, selector))
    print(blog_cnt)
    
blog_cnt()

30


In [57]:
# 페이지 스크롤다운하는 코드
n = 3    # 스크롤할 횟수
i = 0    # 0, 1, 2
while i < n:
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)
    i = i + 1

In [58]:
blog_cnt()

120


# step1. 블로그 글 제목, url 수집

### 블로그 글 제목 수집

In [ ]:
# v3에서는 블로그 글 제목 selector로 class를 더블클릭 후 활용하였다.

In [60]:
# 첫번째 글 제목 수집
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
driver.find_element(By.CSS_SELECTOR, selector).text

'카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천'

In [37]:
# list(range(1, 61, 2))

In [61]:
# 글 제목 여러개를 수집 후 리스트로 저장
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
titles_raw = driver.find_elements(By.CSS_SELECTOR, selector)
# len(titles_raw)

title_list = []
for title_raw in titles_raw:
    title = title_raw.text
    title_list.append(title)
    # print(title)

print(len(title_list))
title_list

120


['카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천',
 '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천',
 '경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함',
 '[경기도 여주] 아이랑 가볼만한 유기농 밀가루 베이커리 카페, 가루영웅베이커리 방문 후기',
 '8개월 아기 안산 나들이 추천_경기도미술관·마리나베이커리·카페미스터신 수유실까지 완벽',
 '[경기도 포천 베이커리 카페]내촌상회',
 '경기도 양주 대형카페 브루다 양주 베이커리 카페',
 '아기랑 경기도 - 의정부 핫한 카페 투하츠 베이커리',
 '경기도 화성에서 찾은 숨겨진 보석, 고향의봄 베이커리 카페',
 '경기도 화성 카페: 고향의봄 베이커리 카페에서 만난 여유',
 '경치 맛집! 경기도 화성 카페, 고향의봄 베이커리 카페 방문 후기',
 '경기도 화성에서 찾은 숨은 보석, 고향의봄 베이커리 카페',
 '[시흥] 경기도 시흥 카페 베이커리가 맛있는 곳 "늘솜당" 서울, 인천근교 로 추천!',
 '경기도 화성에서 찾은 숨은 보석, 고향의봄 베이커리 카페 방문 후기!',
 '[경기도 성남/위례] 주차 가능한 남위례역 베이커리 카페 아우어베이커리 위례점',
 '경기도 화성에서 만난 초대형 베이커리 카페, 고향의봄 베이커리 카페 방문기',
 '경기도 건축문화상 은상 대형카페 페이브 베이커리 화성남양점',
 '경기도 화성에서 찾은 인생 쿠키 맛집, 고향의봄 베이커리 카페',
 '내돈내산 경기도 남양주 아기랑가기 좋은 베이커리카페 하우스 용암(haus yongam)',
 '경기도 화성에서 발견한 숨겨진 보석, 고향의봄 베이커리 카페!',
 '경기도 화성시 포근베이커리 카페/전통찻집 - 달콤한여행',
 '경기도 화성에서 맛있는 디저트 경험, 고향의봄 베이커리 카페',
 '경기도 화성에서 찾은 보물, 고향의봄 베이커리 카페 방문기',
 '경기도 하남시 감일지구 서하남근처 베이커리카페 온더브레드 하남점 브런치 맛집 모임추천 새벽까지 영업',

### 글 url 수집하기

In [ ]:
# .text : 태그 사이에 들어있는 텍스트 추출
# .get_attribute("href") : 태그 안에 들어있는 속성(ex. href url)값 추출

In [62]:
selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
title_raw = driver.find_element(By.CSS_SELECTOR, selector)
# len(titles_raw)
title_raw.get_attribute('href')

'https://blog.naver.com/doin1914/224164042092'

In [63]:
# 글 url 120개 수집하고 리스트로 저장하기
selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
urls_raw = driver.find_elements(By.CSS_SELECTOR, selector)

url_list = []
for url_raw in urls_raw:   
    url = url_raw.get_attribute('href')
    url_list.append(url)
    # print(url)

print(len(url_list))
url_list

120


['https://blog.naver.com/doin1914/224164042092',
 'https://blog.naver.com/jung35745/224163886985',
 'https://blog.naver.com/oifkhlfzio935/224168098738',
 'https://blog.naver.com/dusendigd/224166414878',
 'https://blog.naver.com/sh_note106/224164508577',
 'https://blog.naver.com/comamiyo/224162292781',
 'https://blog.naver.com/sweatlatte46/224160792443',
 'https://blog.naver.com/nalmong126/224165872965',
 'https://blog.naver.com/pjuprhtdpr246/224164850015',
 'https://blog.naver.com/pvdwqbukac710/224166040543',
 'https://blog.naver.com/bemwrlgigc615/224167118923',
 'https://blog.naver.com/duezvaiyqs813/224167030929',
 'https://blog.naver.com/syeon_nolja/224164817653',
 'https://blog.naver.com/dgtshmoozq986/224168127447',
 'https://blog.naver.com/lucky_line_0507/224152689117',
 'https://blog.naver.com/mofdumcesb161/224167065030',
 'https://blog.naver.com/uquiz76ham/224164528406',
 'https://blog.naver.com/kqtczfypyh423/224163611543',
 'https://blog.naver.com/007topkorea/224160426819',
 'ht

In [64]:
# 데이터프레임 저장
df_title_url = pd.DataFrame({"글제목": title_list, "url":url_list })

# csv로 저장
df_title_url.to_csv("title_url.csv", encoding='utf-8-sig')

In [ ]:
# [실습] 검색어를 변경해서 글 url 200개 크롤링 및 데이터프레임, csv 저장.

# step2. 블로그 글 크롤링

In [ ]:
# [실습] 
# 1. 글 class로 css selector 찾기 연습.
# 2. 검색어를 변경해서 글 url 200개 크롤링 및 데이터프레임, csv 저장.

In [16]:
# 검색어
query_txt = "용산 맛집"

In [17]:
# 크롬 드라이버 : 파이썬이 크롬 브라우저를 통제할 수 있게 도와주는 프로그램
# 크롬 드라이버를 '현재 작업 폴더'에 설치
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저
# driver : 크롬 브라우저

driver.get('https://www.naver.com/')
time.sleep(2)    # 2초간 정지

In [18]:
# 네이버 검색창에 "검색어" 검색
selector = "#query"
element = driver.find_element(By.CSS_SELECTOR, selector)    # 검색창 찾기
element.clear()                # 현재 검색창에 무언가 들어있으면 삭제
element.send_keys(query_txt)   # 검색창에 검색어 입력
element.submit()    # 검색 버튼 클릭  
time.sleep(1)

In [19]:
# '블로그' 클릭
driver.find_element(By.LINK_TEXT, "블로그").click( )
time.sleep(1)

In [20]:
driver.find_element(By.LINK_TEXT, "옵션").click( )
time.sleep(1)

In [21]:
driver.find_element(By.LINK_TEXT, "1주").click( )
time.sleep(1)

In [22]:
# 글 개수 확인  👍 def는 “함수를 정의한다
def Yongsan_cnt():
    # 글 CSS SELECTOR 
    selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
    Yongsan_cnt = len(driver.find_elements(By.CSS_SELECTOR, selector))
    print(Yongsan_cnt)
    
Yongsan_cnt()

30


In [23]:
# 페이지 스크롤다운하는 코드
n = 6    # 스크롤할 횟수
i = 0    # 0, 1, 2
while i < n:
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)
    i = i + 1

In [24]:
Yongsan_cnt()

210


In [26]:
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
driver.find_element(By.CSS_SELECTOR, selector).text

'용산 점심 맛집 도야집 가브리 보쌈 정식 한우육회비빔밥 후기'

In [27]:
# 글 제목 여러개를 수집 후 리스트로 저장
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
Yongsan_m = driver.find_elements(By.CSS_SELECTOR, selector)
# len(titles_raw)

Yongsan_list = []
for Yongsan_m in Yongsan_m:
    title = Yongsan_m.text
    Yongsan_list.append(title)
    # print(title)

print(len(  Yongsan_list))
Yongsan_list

210


['용산 점심 맛집 도야집 가브리 보쌈 정식 한우육회비빔밥 후기',
 '용산와인바 용산맛집 데이트 엠버앰버 다녀온 후기 - 와인종류',
 "[용산 이자카야] 신용산 숙성회가 맛있는 용산 맛집 '바다약방'",
 '숙대입구 맛집 저녁 화양연가 용산 양갈비',
 '유럽감성 느좋 용산 샌드위치와 피자맛집 비오피(B.O.P) 용산점',
 "서울 용산 맛집 용산역 이탈리아 가정식 레스토랑 '파브리키친'",
 '용산 피자 맛집 비오피용산점 삼각지역핫플',
 '용리단길맛집 고산식육점 묵은지가 킥인 찐 용산맛도리 흑돼지모둠구이 내돈내산 후기',
 '용산역 근처 맛집 추천 쇼니노 예약 흑백요리사 브런치',
 '[용산] 라자냐 필수 주문인 한남동 맛집 에피세리꼴라주',
 '용산 닭갈비 맛집 │ 오근내2닭갈비 주말 웨이팅 솔직 후기',
 '용산맛집 아투 용산 서울이색데이트 낮에는 네오비스트로 카페 저녁에는 와인바',
 '용산고기집 난로연 남영점 용산 맛집 숙대 남영동 소갈비',
 '용산 한강뷰 레스토랑 아이오유 분위기 맛집 런치코스 혼밥',
 '용산저녁맛집, 용리단길 기념일에 오기 좋은 쇼미더크랩 용산점',
 '용산 텐동 맛집 | 용산 산동 웨이팅 꿀팁 솔직후기',
 '서울드래곤시티인스타일 런치 후기! 서울호텔뷔페 저렴한 용산맛집 추천',
 '용산용리단길맛집 한우 우마카세 프라이빗한 룸식당 고청담 용산점',
 '용산역 맛집 라멘 멘야서울 용산아이파크몰점',
 '[삼각지/용산맛집] 용리단길 데이트 가성비 좋은 회전초밥 맛집 ‘이아초밥 용산점‘',
 '용산 맛집 : 아이파크몰 돼지곰탕 백옥 용산 데이트',
 '용산핫플 용리단길 브런치맛집 도토리',
 '용산역 한식 맛집 빠르쥬 덮밥&강정 포장',
 '용산역맛집 :: 아늑한 유럽 감성 <카토> 디너C세트 후기',
 '바다약방 용산점 이자카야 숙성회 해산물 전통주 용산맛집',
 '용산 쇼니노 흑백요리사 맛집, 예약 주차 꿀팁',
 "서울 용산 딤섬 맛집 '팀호완' / 오픈런해서 원하는 딤섬 골라먹기!",
 '서울 용산 아이파크몰 7층맛집 

In [29]:
#.get_attribute('href') 뜻  👉 HTML 요소의 href 속성 값을 가져와라 라는 의미야.   href=링크주소의미

selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
title_raw = driver.find_element(By.CSS_SELECTOR, selector)
title_raw.get_attribute('href')

'https://blog.naver.com/kiki2a/224165549259'

In [30]:
selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
Yongsan_url = driver.find_elements(By.CSS_SELECTOR, selector)

Yongsan_url_list = []
for Yongsan_url in Yongsan_url:   
    url = Yongsan_url.get_attribute('href')
    Yongsan_url_list.append(url)
    # print(url)

print(len(Yongsan_url_list))
Yongsan_url_list

210


['https://blog.naver.com/kiki2a/224165549259',
 'https://blog.naver.com/kyuriful/224162931556',
 'https://blog.naver.com/iamsjh168/224160450532',
 'https://blog.naver.com/djawndnjs92/224167638901',
 'https://blog.naver.com/lizdoll/224165901287',
 'https://blog.naver.com/js2-log/224165252815',
 'https://blog.naver.com/msunh/224165959782',
 'https://blog.naver.com/pimk5268/224168020466',
 'https://blog.naver.com/offspring84/224164695926',
 'https://blog.naver.com/nemnem22/224109605123',
 'https://blog.naver.com/rlawndud1541/224164064623',
 'https://blog.naver.com/mamstable/224160610499',
 'https://blog.naver.com/doongddal/224165427813',
 'https://blog.naver.com/ong_simi/224162061111',
 'https://blog.naver.com/2004sylee/224166876671',
 'https://blog.naver.com/mmmzzmz/224160037878',
 'https://blog.naver.com/lemoncomet/224163821471',
 'https://blog.naver.com/chandnijo/224166552480',
 'https://blog.naver.com/95thvldk/224160071622',
 'https://blog.naver.com/zpzp0817/224160353547',
 'https://b

In [49]:
# 데이터프레임 저장
df_title_url_aa = pd.DataFrame({"글제목": Yongsan_list, "url":Yongsan_url_list})
df_title_url_aa

,글제목,url
0,용산 점심 맛집 도야집 가브리 보쌈 정식 한우육회비빔밥 후기,https://blog.naver.com/kiki2a/224165549259
1,용산와인바 용산맛집 데이트 엠버앰버 다녀온 후기 - 와인종류,https://blog.naver.com/kyuriful/224162931556
2,[용산 이자카야] 신용산 숙성회가 맛있는 용산 맛집 '바다약방',https://blog.naver.com/iamsjh168/224160450532
3,숙대입구 맛집 저녁 화양연가 용산 양갈비,https://blog.naver.com/djawndnjs92/224167638901
4,유럽감성 느좋 용산 샌드위치와 피자맛집 비오피(B.O.P) 용산점,https://blog.naver.com/lizdoll/224165901287
...,...,...
205,용산 블루리본 커피 맛집 분위기 좋은 카페 폰트,https://blog.naver.com/kkyong620/224160172368
206,용산 횟집 대방어 맛집 울릉집 추천 두번째 방문 후기,https://blog.naver.com/eununmi/224164133592
207,용산 아이파크몰 맛집 낙원타코 내돈내산｜메뉴 추천·가는 방법·예약 꿀팁,https://blog.naver.com/reruns_/224163218323
208,용산 삼각지 맛집 삼각지 노포 맛집 옛집 국수,https://blog.naver.com/korealeejhk/224166499203


In [51]:
# csv로 저장
df_title_url_aa.to_csv("title_url.csv", encoding='utf-8-sig', index=False)

In [ ]:
크롤링

In [ ]:
df = pd.read_csv("title_url.csv")

print(df.shape)
df.head()

In [ ]:
# 첫번째 글 제목
df['글제목'][0]

In [ ]:
# 첫번째 글의 url
i = 0
url = df['url'][i]
url

In [ ]:
# 크롬 브라우저 띄우기
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1)

In [ ]:
# iframe 접근  👉 **웹페이지 안에 들어있는 “또 다른 웹페이지”**야.
driver.switch_to.frame('mainFrame')

In [ ]:
#게시글(제목,내용,글쓴이,날짜 등) 크롤링 후 저장할 공간
target_info = {}

# 블로그 글 제목 첫번째 크롤링
selector = "#SE-308d4d8e-fccc-11f0-925e-8135ef4f9fb3"
title_raw = driver.find_element(By.CSS_SELECTOR, selector)          # title
title = title_raw.text
print(title)

#딕셔너리에 데이터 저장
target_info['title'] = title
target_info

In [ ]:
#글쓴이 크롤링 시작
selector = ".nick"
nickname = driver.find_element(By.CSS_SELECTOR, selector).text

#딕셔너리에 데이터 저장
target_info['nickname'] = nickname
target_info

In [ ]:
#날짜 실습 